In [ ]:
%cd ../..
# %pip install .
import time
from os import environ
# offset = Uniform(-1, 1) * Range(90, 180) deg

# ego = Car on drivableRoad,
#         facing offset relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# otherCar = Car on visible road,
#             facing Range(-15, 15) deg relative to roadDirection

# require (distance from ego to otherCar) < 10

In [ ]:
environ['AP_PORT'] = str(input('port')) # README command uses port=25432

In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as otherCar
join Cameras as cam on t1.cameraId = Cameras.id
where
  TODO: ego facing relative to road direction????
  DISTANCE(cam.egoTranslation, otherCar.trajCentroids, cam.timestamp) < 50 AND
  viewAngle(otherCar.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  contained(otherCar.traj, road_segment_type("visibleRoad"), cam.timestamp) and
  facingRelative(otherCar.heading, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) >= -15 AND
  facingRelative(otherCar.heading, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) < 15 AND
  DISTANCE(cam.egoTranslation, otherCar.centroid, cam.timestamp) < 10 AND
"""

In [ ]:
### Prepare the world for queries ###
from apperception.world import empty_world
import numpy as np
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
world = world.filter(lambda obj: obj.object_type == 'vehicle.car')
random_sample = np.random.uniform(-1,1)
facing_range = (random_sample*90, random_sample*180) 
def pred(otherCar, cam):
    return (
        # TODO: ego facing `offset` relative to road direction??
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and 
        F.view_angle(obj1, cam.ego, cam.timestamp) < 67.5 and 
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) > {facing_range[0]} and 
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) < {facing_range[1]} and 
        F.contained(cam.ego, F.road_segment('road')) and 
        F.contained(obj1.traj, F.road_segment('road'), cam.timestamp) and 
        F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) >= -15 and 
        F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) < 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 10
    )
world = world.filter("lambda obj1, cam: " + 
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
        "F.view_angle(obj1, cam.ego, cam.timestamp) < 67.5 and " +
        f"F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) > {facing_range[0]} and " +
        f"F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) < {facing_range[1]} and " +
        "F.contained(cam.ego, F.road_segment('road')) and " +
        "F.contained(obj1.traj, F.road_segment('road'), cam.timestamp) and " +
        "F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) >= -15 and " +
        "F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) < 15 and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 10")

start = time.time()
# keys = world.get_traj_key()
id_time_camId_filename = world.get_id_time_camId_filename(num_joined_tables=1)

end = time.time()
print(format(end-start))